In [6]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [7]:
import se3_diffuse.utils as du

In [8]:
from data_rigid_diffuser import so3_diffuser
from data_rigid_diffuser import r3_diffuser
from data_rigid_diffuser import oneHot_diffuser
from scipy.spatial.transform import Rotation
from data_rigid_diffuser import rigid_utils as ru
import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs

In [9]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [10]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

# #find better way to incorporate coord_scale

# #needed
# N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
# C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [11]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split

#mode for tablet
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 256
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [12]:
B = 32
L=65
limit = 128
h4_trainData = Helix4_Dataset(coords_tog[:limit])
h4_valData = Helix4_Dataset(coords_apa[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)

In [13]:
B = 32
L=65
limit = 128
prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=128,
              n_atoms=5, coord_div=10, cast_type=torch.float32)
train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)

In [14]:
testiter = iter(train_dL)
bb_dict = next(testiter)

In [15]:
from data_rigid_diffuser import diffuser

In [16]:
config_path='data_rigid_diffuser/base.yaml'

        


In [17]:
fnd = FrameDiffNoise(config_path)


In [18]:
noised_dict = fnd.forward(bb_dict,t_vec=np.ones((B,))*0.03)
#edges are Batch, Length, KNN, 3, bool  
#each edge represents real adj, fake dist

 #real and direct connect, real and non-direct, false and direct # no false and not direct

/mnt/c/Users/nwood/OneDrive/Desktop/gudiff/data_rigid_diffuser/diffuser.py:209: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/torch/csrc/utils/tensor_new.cpp:245.)
  node_type_noised = torch.tensor([self.ohd.forward_marginal(expand_node_type[i].numpy(),t)[0] for i,t in enumerate(t_vec)])


In [19]:
def circular_pe_encoding(n_nodes=128,embed_dim=12, cast_type=torch.float32):
    #positional encoding of node, all repeat every n_nodes
    i_array = np.arange(1,(embed_dim/2)+1)
    period = 2*np.pi
    wk = period/(n_nodes)*i_array**2
    t_array = np.arange(n_nodes)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    
    return pe

#test code
#n_nodes = 6
#src_start = torch.arange(n_nodes) #positive direction
#dst_start = torch.roll(src_start,-1) #negative dir
#xyz = torch.arange(n_nodes)[:,None].repeat((1,3))
#xyz[-1] = torch.tensor([4,4,4])
#xyz[0] = torch.tensor([1,1,1])
#mask = torch.tensor([False,True,True,True,True,False])
#mask = torch.tensor([False,True,False,True,True,False])


def monomer_knngraph(xyz,real_mask, k=2):
    """Takes in array of XYZ coordinates with null nodes repeated at termini.
    
    #Parameters
    real_mask: mask determining residues of real and null nodes
    
    Returns Graph that has a linear connections between all nodes indices"""
    
    n_nodes = xyz.shape[0]
    src_start = torch.arange(n_nodes) #positive direction
    dst_start = torch.roll(src_start,-1) #negative dir
    
    #ma
    real_nodes_src = src_start[real_mask==True]
    null_nodes_src = src_start[real_mask==False]
    real_nodes_dst = dst_start[real_mask==True]
    null_nodes_dst = dst_start[real_mask==False]

    g_knn=dgl.knn_graph(xyz[real_mask],k=k,exclude_self=True) # normal graph
    g_knn.add_nodes(len(null_nodes_src)) # null nodes to the end

    src_knn_shift = torch.zeros_like(src_start)
    dst_knn_shift = torch.zeros_like(dst_start)

    src_knn_shift[:len(real_nodes_src)] = real_nodes_src
    src_knn_shift[len(real_nodes_src):] = null_nodes_src
    dst_knn_shift[:len(real_nodes_src)] = real_nodes_dst
    dst_knn_shift[len(real_nodes_src):] = null_nodes_dst

    g_nodes_index = src_start#torch.arange(n_nodes) 
    src = g_nodes_index[src_knn_shift]
    dst = g_nodes_index[dst_knn_shift]
    
    knn_src, knn_dst = g_knn.edges()
    for i,nullnode in enumerate(null_nodes_src):
        knn_src[knn_src==null_nodes_src[i]] = real_nodes_src[i]
        knn_dst[knn_dst==null_nodes_dst[i]] = real_nodes_dst[i]
    
    g = dgl.graph((knn_src,knn_dst))

    #can just add edges on top, simple to delete parallel; 
    g.add_edges(src,dst) 
    g.add_edges(dst,src)
    g = dgl.to_simple(g)

    return g

def decirc(graph_in):
    """Remove circular connections. Last node to first node"""
    src,dst = graph_in.edges()

    circfw = (src==graph_in.nodes().max()) & (dst==0) 
    circbw = (src==0) & (dst==graph_in.nodes().max())
    both_circ = circfw | circbw
    
    src_out = src[~both_circ]
    dst_out = dst[~both_circ]
    
    return dgl.graph((src_out,dst_out))

def concat_monomer_graphs(graph_list):
    """Add single connection between each graph"""
    
    batched_graph = dgl.batch([decirc(g) for g in graph_list])
    src = torch.zeros((len(graph_list),),dtype=torch.int64) 
    dst = torch.zeros_like(src)
    
    #recirc concats
    nindex = 0
    src[nindex] = 0
    dst[nindex] = len(batched_graph.nodes())-1 #
    
    
    for i in range(1,len(graph_list)):
        src[i] = graph_list[nindex].nodes().max()+nindex
        dst[i] = src[i]+1
        nindex = src[i]
    
    
    batched_graph.add_edges(src,dst)
    batched_graph.add_edges(dst,src)
    
    return batched_graph


In [20]:
#this should probably be already transferred to GPU for speed
class Make_nullKNN_MP_Graphs():
    
    #8 long positional encoding
    NODE_FEATURE_DIM_0 = 12
    EDGE_FEATURE_DIM = 1 # 0 or 1 primary seq connection or not
    NODE_FEATURE_DIM_1 = 2
    
    def __init__(self, KNN=30, mp_stride=4, n_nodes=128, coord_div=10, 
                       cast_type=torch.float32, channels_start=32,
                       ndf1=6, ndf0=32,cuda=True):
        
        self.KNN = KNN
        self.n_nodes = n_nodes
        self.pe = circular_pe_encoding(n_nodes=n_nodes,embed_dim=12, cast_type=torch.float32)
        self.mp_stride = mp_stride
        self.cast_type = cast_type
        self.channels_start = channels_start
        
        self.cuda = cuda
        self.ndf1 = ndf1 #awkard adding of nodes features to mpGraph
        self.ndf0 = ndf0
        
        
    def create_and_batch(self, bb_dict):
        
        graphList = []
        mpGraphList = []
        mpRevGraphList = []
        mpSelfGraphList = []
        
        for j, caXYZ in enumerate(bb_dict['bb_noised']['CA']):
            #round nodes to be real (1) or (null 0)
            #get null node indices from mask
            real_nodes_feats = torch.round(noised_dict['real_nodes_noise'][0]).clamp(0,1)
            real_nodes_mask = real_nodes_feats.sum(-1)>2.99
            print(real_nodes_mask)
            
            graph = monomer_knngraph(caXYZ, real_nodes_mask, k=self.KNN)
            graph.ndata['pe'] = torch.cat((self.pe,real_nodes_feats),dim=-1)
            graph.ndata['pos'] = caXYZ
            graph.ndata['bb_ori'] = torch.cat((bb_dict['bb_noised']['N_CA'][j],  bb_dict['bb_noised']['C_CA'][j]),axis=1)
            
            #gather edge data from all possible noised edges produced
            gsrc, gdst = graph.edges()
            num_edges = len(gsrc)
            
            #adjacent AA are one apart, or the loop connection from zero node to the last node
            adj_nodes_mask = ((torch.abs(gsrc-gdst)==1) | (torch.abs(gsrc-gdst)==len(gsrc)-1)) 

            
            
            #actually we need to determine
            null_nodes = torch.arange(self.n_nodes)[~real_nodes_mask]
            real_nodes = torch.arange(self.n_nodes)[real_nodes_mask]
            graph.ndata['real_nodes'] = ~real_nodes_mask
            graph.ndata['null_nodes'] =  real_nodes_mask

            #broadcast each src/dst node against all null nodes, if any match along node dimension is a null edge
            gsrc_compare = (gsrc[:,None] - null_nodes[None,:])
            gdst_compare = (gdst[:,None] - null_nodes[None,:])
            null_edges_src_ind = torch.where(gsrc_compare==0,True,False).any(dim=1)
            null_edges_dst_ind = torch.where(gdst_compare==0,True,False).any(dim=1)
            
            #broadcast each src/dst node against all real or null nodes, if any match along node dimension is a null edge
            gsrc_compare = (gsrc[:,None] - real_nodes[None,:])
            gdst_compare = (gdst[:,None] - real_nodes[None,:])
            real_edges_src_ind = torch.where(gsrc_compare==0,True,False).any(dim=1)
            real_edges_dst_ind = torch.where(gdst_compare==0,True,False).any(dim=1)

            null_edges = null_edges_src_ind | null_edges_dst_ind
            real_edges = real_edges_src_ind & real_edges_dst_ind
            adj_real = adj_nodes_mask&real_edges
            
            ###
            enoised = noised_dict['edge_cons'][j]
            enoised = enoised.reshape((-1,EDGE_DIM,2))[:num_edges]
            edata = torch.zeros(gsrc.shape+(EDGE_DIM,))
            oh1_mask = torch.tensor([1,0,0]).unsqueeze(-1)
            oh2_mask = torch.tensor([0,1,0]).unsqueeze(-1)
            oh3_mask = torch.tensor([0,0,1]).unsqueeze(-1)
            
            
            #needs to add - /+ N/C directon for this data
            
            
            edata[real_edges] = torch.gather(enoised[real_edges],2,oh2_mask.repeat(real_edges.sum(),1,1))[:,:,0]
            edata[adj_real] = torch.gather(enoised[adj_real],2,oh1_mask.repeat(adj_real.sum(),1,1))[:,:,0]
            edata[null_edges] = torch.gather(enoised[null_edges],2,oh3_mask.repeat(null_edges.sum(),1,1))[:,:,0]
            
            
            graph.edata['con'] = edata
            
            #way to get real_edges and null edges in loop code
            # null_edges = torch.zeros_like(gsrc)
            # for i,x in enumerate(gsrc):
            #     if gsrc[i] in null_nodes or gdst[i] in null_nodes:
            #         null_edges[i] = 1

            # real_edges = torch.zeros_like(gsrc)
            # for i,x in enumerate(gsrc):
            #     if gsrc[i] in real_nodes and gdst[i] in real_nodes:
            #         real_edges[i] = 1

            return graph, real_nodes, null_nodes
        
        

In [21]:

nkg = Make_nullKNN_MP_Graphs()
EDGE_DIM = 3
g,rn,nn =nkg.create_and_batch(noised_dict)

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False])

In [22]:
ge = g.edata['con']

In [32]:
noised_dict['edge_cons'][0].shape

torch.Size([128, 30, 3, 2])

In [21]:
torch.round(ge[:,0:2]).type(torch.bool).any(dim=1).sum()

tensor(1571)

In [22]:
torch.round(ge[:,0].unsqueeze(-1)).sum()

tensor(128.)

In [23]:
#remake graph based on noised_edge data, should this data be on the nodes?
real_nodes_edges = torch.round(ge[:,0].unsqueeze(-1)).any(dim=1) #grab the adjacent real nodes from edge data
src, dst = g.edges()
real_nodes=src[real_nodes_edges].unique() #find real nodes form edges

combined = torch.cat((real_nodes, torch.arange(nkg.n_nodes)))
uniques, counts = combined.unique(return_counts=True)
null_nodes = uniques[counts == 1]

real_mask = torch.zeros((nkg.n_nodes,),dtype=torch.bool)
real_mask[real_nodes]=1

#make graph again based on the edge data pulled

monomer_knngraph(g.ndata['pos'],real_mask, k=30)



Graph(num_nodes=128, num_edges=1699,
      ndata_schemes={}
      edata_schemes={'count': Scheme(shape=(), dtype=torch.int64)})

In [39]:
torch.ones_like(fnd.mask_shift.unsqueeze(-1)).repeat((1,)*len(fnd.mask_shift.shape)+(5,)).shape

torch.Size([32, 128, 5])

In [38]:
fnd.mask_shift.shape

torch.Size([32, 128])

In [131]:
src[real_nodes].unique()

tensor([34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
        70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
        88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98])

In [84]:
fuck should I be predicting null/read nodes only... fuck


In [85]:
src

tensor([  0,   0,   1,  ..., 126, 127, 127])

In [ ]:
def graph_from_noised_edges(prev_graph,)

In [54]:
#real and direct connect, [1,0,0]
#real and non-direct, [0,1,0]
#false and direct # no false and not direct[0,0,1]
np.round(ge).clamp(0,1)

tensor([[-0., 1., 0.],
        [-0., 1., 0.],
        [0., 1., -0.],
        ...,
        [-0., 1., -0.],
        [-0., 1., 0.],
        [-0., 1., -0.]])

In [43]:
def softmax(xs):
    return np.exp(xs) / sum(np.exp(xs))

In [ ]:
#now we need to do midpoint graph:
#we need another object to make a graph from the noised edges
#instead of pulling from enoised pull from raw onehots
#during inference time just pull from edges to start


#i think knn graph but but with null nodes group as 1 (up to make connected as 1?)

In [20]:
class Make_KNN_MP_Graphs():
    
    #8 long positional encoding
    NODE_FEATURE_DIM_0 = 12
    EDGE_FEATURE_DIM = 1 # 0 or 1 primary seq connection or not
    NODE_FEATURE_DIM_1 = 2
    
    def __init__(self, mp_stride=4, n_nodes=65, radius=15, coord_div=10, cast_type=torch.float32, channels_start=32,
                       ndf1=6, ndf0=32,cuda=True):
        
        self.KNN = 30
        self.n_nodes = n_nodes
        self.pe = make_pe_encoding(n_nodes=n_nodes)
        self.mp_stride = mp_stride
        self.cast_type = cast_type
        self.channels_start = channels_start
        
        self.cuda = cuda
        self.ndf1 = ndf1 #awkard adding of nodes features to mpGraph
        self.ndf0 = ndf0
        
    def create_and_batch(self, bb_dict):
        
        graphList = []
        mpGraphList = []
        mpRevGraphList = []
        mpSelfGraphList = []
        
        for j, caXYZ in enumerate(bb_dict['CA']):
            graph = dgl.knn_graph(caXYZ, self.KNN)
            graph.ndata['pe'] = self.pe
            graph.ndata['pos'] = caXYZ
            graph.ndata['bb_ori'] = torch.cat((bb_dict['N_CA'][j],  bb_dict['C_CA'][j]),axis=1)
            
            #define covalent connections
            esrc, edst = graph.edges()
            graph.edata['con'] = (torch.abs(esrc-edst)==1).type(self.cast_type).reshape((-1,1))
            
            mp_list = torch.zeros((len(list(range(0,self.n_nodes, self.mp_stride))),caXYZ.shape[1]))
            
            new_src = torch.tensor([],dtype=torch.int)
            new_dst = torch.tensor([],dtype=torch.int)
            
            new_src_rev = torch.tensor([], dtype=torch.int)
            new_dst_rev = torch.tensor([], dtype=torch.int)
           
            i=0#mp list counter
            for x in range(0,self.n_nodes, self.mp_stride):
                src, dst = graph.in_edges(x) #dst repeats x
                n_tot = torch.cat((torch.tensor(x).unsqueeze(0),src)) #add x to node list
                mp_list[i] = caXYZ[n_tot].sum(axis=0)/n_tot.shape[0]
                mp_node = i + graph.num_nodes() #add midpoints nodes at end of graph
                #define edges between midpoint nodes and nodes defining midpoint for midpointGraph
                
                new_src = torch.cat((new_src,n_tot))
                new_dst = torch.cat((new_dst,
                                     (torch.tensor(mp_node).unsqueeze(0).repeat(n_tot.shape[0]))))
                #and reverse graph for coming off
                new_src_rev = torch.cat((new_src_rev,
                                         (torch.tensor(mp_node).unsqueeze(0).repeat(n_tot.shape[0]))))
                new_dst_rev = torch.cat((new_dst_rev,n_tot))
                
                i+=1
                
            mpGraph = dgl.graph((new_src,new_dst))
            mpGraph.ndata['pos'] = torch.cat((caXYZ,mp_list),axis=0).type(self.cast_type)
            mp_node_indx = torch.arange(0,self.n_nodes, self.mp_stride).type(torch.int)
            #match output shape of first transformer
            pe_mp = torch.cat((self.pe,torch.zeros((self.pe.shape[0],self.channels_start-self.pe.shape[1]))),axis=1)
            mpGraph.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph.edata['con'] = torch.zeros((mpGraph.num_edges(),1))
            
            mpGraph_rev = dgl.graph((new_src_rev,new_dst_rev))
            mpGraph_rev.ndata['pos'] = torch.cat((caXYZ,mp_list),axis=0).type(self.cast_type)
            mpGraph_rev.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph_rev.edata['con'] = torch.zeros((mpGraph_rev.num_edges(),1))
            
            #make graph for self interaction of midpoints
            v1,v2,edge_data, ind = define_graph_edges(len(mp_list))
            mpSelfGraph = dgl.graph((v1,v2))
            mpSelfGraph.edata['con'] = edge_data.reshape((-1,1))
            mpSelfGraph.ndata['pe'] = self.pe[mp_node_indx] #not really needed
            mpSelfGraph.ndata['pos'] = mp_list.type(self.cast_type)
            
            
            mpSelfGraphList.append(mpSelfGraph) 
            mpGraphList.append(mpGraph)
            mpRevGraphList.append(mpGraph_rev)
            graphList.append(graph)
        
        return dgl.batch(graphList), dgl.batch(mpGraphList), dgl.batch(mpSelfGraphList), dgl.batch(mpRevGraphList)
    
    def prep_for_network(self, bb_dict, cuda=True):
    
        batched_graph, batched_mpgraph, batched_mpself_graph, batched_mpRevgraph =  self.create_and_batch(bb_dict)
        
        edge_feats        =    {'0':   batched_graph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
        edge_feats_mp     = {'0': batched_mpgraph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]} #def all zero now
        edge_feats_mpself = {'0': batched_mpself_graph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
#         edge_feats_mp     = {'0': batched_mpRevgraph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
        batched_graph.edata['rel_pos']   = _get_relative_pos(batched_graph)
        batched_mpgraph.edata['rel_pos'] = _get_relative_pos(batched_mpgraph)
        batched_mpself_graph.edata['rel_pos'] = _get_relative_pos(batched_mpself_graph)
        batched_mpRevgraph.edata['rel_pos'] = _get_relative_pos(batched_mpRevgraph)
        # get node features
        node_feats =         {'0': batched_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM_0, None],
                              '1': batched_graph.ndata['bb_ori'][:,:self.NODE_FEATURE_DIM_1, :3]}
        node_feats_mp =      {'0': batched_mpgraph.ndata['pe'][:, :self.ndf0, None],
                              '1': torch.ones((batched_mpgraph.num_nodes(),self.ndf1,3))}
        #unused
        node_feats_mpself =  {'0': batched_mpself_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM_0, None]}
        
        if cuda:
            bg,nf,ef = to_cuda(batched_graph), to_cuda(node_feats), to_cuda(edge_feats)
            bg_mp, nf_mp, ef_mp = to_cuda(batched_mpgraph), to_cuda(node_feats_mp), to_cuda(edge_feats_mp)
            bg_mps, nf_mps, ef_mps = to_cuda(batched_mpself_graph), to_cuda(node_feats_mpself), to_cuda(edge_feats_mpself)
            bg_mpRev = to_cuda(batched_mpRevgraph)
            
            return bg,nf,ef, bg_mp, nf_mp, ef_mp, bg_mps, nf_mps, ef_mps, bg_mpRev
        
        else:
            bg,nf,ef = batched_graph, node_feats, edge_feats
            bg_mp, nf_mp, ef_mp = batched_mpgraph, node_feats_mp, edge_feats_mp
            bg_mps, nf_mps, ef_mps = batched_mpself_graph, node_feats_mpself, edge_feats_mpself
            bg_mpRev = batched_mpRevgraph
            
            return bg,nf,ef, bg_mp, nf_mp, ef_mp, bg_mps, nf_mps, ef_mps, bg_mpRev
        
            

def get_edge_features(graph,edge_feature_dim=1):
    return {'0': graph.edata['con'][:, :edge_feature_dim, None]}

def define_poolGraph(n_nodes, batch_size, cast_type=torch.float32, cuda_out=True ):
    
    v1,v2,edge_data, ind = define_graph_edges(n_nodes)
    #pe = make_pe_encoding(n_nodes=n_nodes)#pe e
    
    graphList = []
    
    for i in range(batch_size):
        
        g = dgl.graph((v1,v2))
        g.edata['con'] = edge_data.type(cast_type).reshape((-1,1))
        g.ndata['pos'] = torch.zeros((n_nodes,3),dtype=torch.float32)

        graphList.append(g)
        
    batched_graph = dgl.batch(graphList)
    
    if cuda_out:
        return to_cuda(batched_graph)
    else:
        return batched_graph

In [35]:
xyz

tensor([[1, 1, 1],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4],
        [4, 4, 4]])

In [64]:
dst

tensor([2, 3, 4, 5, 1, 0])